# Spaced Repetition Helper


This notebook manages a simple spaced-repetition schedule in a CSV file.

In [ ]:
import pandas as pd
from datetime import date, timedelta

csv_path = 'spaced_cards.csv'
try:
    df = pd.read_csv(csv_path, parse_dates=['next_review'])
except FileNotFoundError:
    df = pd.DataFrame(columns=['id', 'question', 'answer', 'interval_days', 'next_review'])
    df.to_csv(csv_path, index=False)
df.head()

In [ ]:
def add_card(question, answer, interval_days=1):
    global df
    today = date.today()
    next_review = today + timedelta(days=interval_days)
    new = {
        'id': len(df),
        'question': question,
        'answer': answer,
        'interval_days': interval_days,
        'next_review': next_review,
    }
    df = pd.concat([df, pd.DataFrame([new])], ignore_index=True)
    df.to_csv(csv_path, index=False)

# Example:
# add_card("State Newton's 2nd law.", r"$\\vec F = m\\vec a$")

In [ ]:
today = date.today()
due = df[df['next_review'] <= pd.to_datetime(today)]
due[['id', 'question', 'answer']]

In [ ]:
def mark_reviewed(card_id, rating):
    '''rating: 0 = hard, 1 = okay, 2 = easy'''
    global df
    row = df.loc[df['id'] == card_id].iloc[0]
    if rating == 0:
        new_interval = 1
    elif rating == 1:
        new_interval = max(1, int(row['interval_days'] * 1.5))
    else:
        new_interval = max(2, int(row['interval_days'] * 2.5))

    df.loc[df['id'] == card_id, 'interval_days'] = new_interval
    df.loc[df['id'] == card_id, 'next_review'] = date.today() + timedelta(days=new_interval)
    df.to_csv(csv_path, index=False)

# Example usage after testing yourself:
# mark_reviewed(card_id=0, rating=2)